In [1]:
########################################
import os
import re
import json
import random
import pickle
import operator
import itertools

import numpy
import sklearn.ensemble
import sklearn.linear_model
import sklearn.feature_extraction
import pandas
import html
#import raha
########################################

In [2]:
import os
import re
import io
import sys
import math
import json
import html
import pickle
import difflib
import unicodedata
import bs4
import bz2
import py7zr
import numpy
import mwparserfromhell
import libarchive.public
#import dataset
import wikitextparser as wtp
from pprint import pprint
from wikitextparser import remove_markup, parse
import datetime
from pandas import DataFrame
import random
import pickle
import pandas as pd
import logging
from datasets import *
import pandas

In [3]:
from gensim.models import FastText

In [4]:
import fasttext

In [5]:
def value_normalizer(value):
        """
        This method takes a value and minimally normalizes it.
        """
        value = html.unescape(value)
        value = re.sub("[\t\n ]+", " ", value, re.UNICODE)
        value = value.strip("\t\n ")
        return value

In [6]:
def read_csv_dataset(dataset_path):
        """
        This method reads a dataset from a csv file path.
        """
        dataframe = pandas.read_csv(dataset_path, sep=",", header="infer", encoding="utf-8", dtype=str,
                                    keep_default_na=False, low_memory=False).applymap(value_normalizer)
        return dataframe


In [8]:
dataset_name = "hospital"
dataset_dictionary = {
        "name": dataset_name,
        "path": os.path.abspath(os.path.join( "datasets", dataset_name, "dirty.csv")),
        "clean_path": os.path.abspath(os.path.join("datasets", dataset_name, "clean.csv"))
    }

In [9]:
flight_fds=[['flight','act_dep_time'],['flight','sched_dep_time'],['flight','sched_arr_time'],['flight','act_arr_time']]
flight_fds_att=['flight','act_dep_time','sched_dep_time','sched_arr_time','act_arr_time']
domain_dirty_col=['address_1','city','state','county','state','area_code']
domain_clean_col=['Address1','City','State','CountyName','city','state','area_code']
tax_fds=[['zip','city']]

In [10]:
name = dataset_dictionary["name"]
path = dataset_dictionary["path"]
dataframe = read_csv_dataset(dataset_dictionary["path"])
if "clean_path" in dataset_dictionary:
    has_ground_truth = True
    clean_path = dataset_dictionary["clean_path"]
    clean_dataframe = read_csv_dataset(dataset_dictionary["clean_path"])
if "repaired_path" in dataset_dictionary:
    has_been_repaired = True
    repaired_path = dataset_dictionary["repaired_path"]
    repaired_dataframe = read_csv_dataset(dataset_dictionary["repaired_path"])

In [8]:
#dataframe['zip']=clean_dataframe['zip']

In [18]:
repaired=((u'__label__food-safety', u'__label__baking', u'__label__equipment', u'__label__substitutions', u'__label__bread'), array([0.0857 , 0.0657, 0.0454, 0.0333, 0.0333]))

NameError: name 'array' is not defined

In [11]:
retrain_data=prepare_dataset_for_retrain_realworld(clean_dataframe, dataframe)

In [12]:
 predict_error=retrain_data.iloc[5]

In [13]:
 row=list(filter(None, predict_error))

In [14]:
row

['6',
 '10018',
 'callahan eye foundation hospital',
 '1720 university blvd',
 'empty',
 'empty',
 'birmingham',
 'al',
 '35233',
 'jefferson',
 '2053258100',
 'acute care hospitals',
 'voluntary non-profit - private',
 'yes',
 'surgical infection prevention',
 'scip-inf-6',
 'surgery patients needing hair removed from the surgical area before surgery who had hair removed using a safer method (electric clippers or hair removal cream c not a razor)',
 'empty',
 'empty',
 'al_scip-inf-6']

In [15]:
predicted=retrain_data[['zip','city']]

In [16]:
predicted.iloc[1].tolist()

['35233', 'birmingham']

In [20]:
total_error=calculate_total_error_realworld(clean_dataframe, dataframe)

In [21]:
total_error

509

In [12]:
#clean_dataframe

# Hospital_Fasttext

In [17]:
fds_hos=[["zip", "city"], ["zip", "state"], ["zip", "county"]]

In [22]:
total_error_to_repaired=0
total_repaired=0
count=211636
for fds in  fds_hos:
    count=count+1
    train_data_rows=[]
    data_for_retrain=retrain_data[fds]
    #print(data_for_retrain[fds[0]])
    #data_for_retrain[fds[0]]=[s for s in data_for_retrain[fds[0]] if s]
    #print(data_for_retrain[fds[0]])
    data_for_retrain[fds[1]]='__label__' + data_for_retrain[fds[1]].astype(str)
    data_for_retrain[fds[1]]=[s.replace(" ", "_") for s in data_for_retrain[fds[1]]]
    df3 = data_for_retrain[data_for_retrain[fds[1]] != "__label__None"]
    #df3 = df3[df3[fds[0]] != ]
    #f3 = df3[df3[fds[0]] != "None"]
    #model = fasttext.train_supervised(input=df3)
    df3.to_csv(r'datasets/hospital_tax.txt', index=False, sep=' ', header=False)
    ! wc datasets/hospital_tax.txt
    #! head -n 819  datasets/training_flight.txt > datasets/flight.train
    #! tail -n 200  datasets/training_flight.txt > datasets/flight.valid
    model_flight=fasttext.train_supervised(input="datasets/hospital_tax.txt",epoch=100)
    model_name="hospital_"+str(count)+"_"
    model_flight.save_model(model_name+".bin")
    error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,[fds[1]])
    #model_flight.test("datasets/flight.valid")
    #print_results(*model_flight.test('datasets/flight.valid'))
    for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        predict_value=retrain_data.at[index, fds[0]] #retrain withdraw
        if predict_value:
            repaired=model_flight.predict(predict_value,k=1,threshold=0.5)
            try:
                if repaired[0][0]:
                    total_error_to_repaired=total_error_to_repaired+1   
                    k=repaired[0][0].replace('__label__','')
                    repaired_k=k.replace('_',' ')
                    #print(repaired_k,actual_value)
                    repaired_k=repaired_k.strip()
                    actual_value=actual_value.strip()
                    if repaired_k==actual_value:
                        total_repaired=total_repaired+1
            except:
                print("Threshold")
    print(total_error,total_error_to_repaired,total_repaired )
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)


  967  1905 23046 datasets/hospital_tax.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


509 32 32
  974  1918 17382 datasets/hospital_tax.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


509 58 58
  961  1892 22197 datasets/hospital_tax.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


509 97 97
509 97 97


NameError: name 'evaluate_model' is not defined

In [25]:
repaired[1][0]

0.9971268177032471

In [15]:
def evaluate_model (total_error, total_error_to_repair, total_correction):
        if total_error_to_repair==0:
            precision=0.00
        else:
            precision=total_correction/total_error_to_repair
            precision=round(precision,2)
        if total_error==0:
            recall=0.00
        else:
            recall=total_correction/total_error
            recall=round(recall,2)
        if (precision+recall)==0:
            f_score=0.00
        else:
            f_score=(2 * precision * recall) / (precision + recall) 
            f_score=round(f_score,2)     
        return precision, recall,f_score


# Tax Training

In [36]:
total_error_to_repaired=0
total_repaired=0
count=210944
for fds in  tax_fds:
    count=count+1
    train_data_rows=[]
    data_for_retrain=retrain_data[fds]
    #print(data_for_retrain[fds[0]])
    #data_for_retrain[fds[0]]=[s for s in data_for_retrain[fds[0]] if s]
    #print(data_for_retrain[fds[0]])
    data_for_retrain[fds[1]]='__label__' + data_for_retrain[fds[1]].astype(str)
    data_for_retrain[fds[1]]=[s.replace(" ", "_") for s in data_for_retrain[fds[1]]]
    df3 = data_for_retrain[data_for_retrain[fds[1]] != "__label__None"]
    #df3 = df3[df3[fds[0]] != ]
    #f3 = df3[df3[fds[0]] != "None"]
    #model = fasttext.train_supervised(input=df3)
    df3.to_csv(r'datasets/training_tax.txt', index=False, sep=' ', header=False)
    ! wc datasets/training_tax.txt
    #! head -n 819  datasets/training_flight.txt > datasets/flight.train
    #! tail -n 200  datasets/training_flight.txt > datasets/flight.valid
    model_flight=fasttext.train_supervised(input="datasets/training_tax.txt",epoch=100)
    #model_name="tax_"+str(count)+"_"
    model_flight.save_model("tax_zip_city.bin")
    error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,fds)
    #model_flight.test("datasets/flight.valid")
    #print_results(*model_flight.test('datasets/flight.valid'))
    for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        predict_value=retrain_data.at[index, fds[0]] #retrain withdraw
        if predict_value:
            repaired=model_flight.predict(predict_value,k=1,threshold=0.5)
            try:
                if repaired[0][0]:
                    total_error_to_repaired=total_error_to_repaired+1   
                    k=repaired[0][0].replace('__label__','')
                    repaired_k=k.replace('_',' ')
                    #print(repaired_k,actual_value)
                    repaired_k=repaired_k.strip()
                    actual_value=actual_value.strip()
                    if repaired_k==actual_value:
                        total_repaired=total_repaired+1
            except:
                print("Threshold")
    print(total_error,total_error_to_repaired,total_repaired )
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)

            

/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


 199800  399600 4869093 datasets/training_tax.txt
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
Threshold
89908 119 119
89908 119 119
1.0 0.0 0.0


In [33]:
model = fasttext.load_model('tax_1333_.bin')

In [30]:
fds=['zip','city']

In [18]:
error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,fds)

In [34]:
total_error_to_repaired=0
total_repaired=0
totaL=0
for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
    predict_value=retrain_data.at[index, fds[0]]
    totaL=totaL+1
    print(predict_value)

    if predict_value:

   # total_error_to_repaired=total_error_to_repaired+1

        repaired=model.predict(predict_value,k=1)
        print(repaired)
        try:
            if repaired[0][0]:
                total_error_to_repaired=total_error_to_repaired+1   
                k=repaired[0][0].replace('__label__','')
                repaired_k=k.replace('_',' ')
                #print(repaired_k,actual_value)
                repaired_k=repaired_k.strip()
                actual_value=actual_value.strip()
                if repaired_k==actual_value:
                    total_repaired=total_repaired+1
        except Exception as e :
            print("Threshold", str(e))
    #print(total_error,total_error_to_repaired,total_repaired )
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,totaL,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)

18403
(('__label__SAN_FRANCISCO',), array([0.00053957]))
83212
(('__label__SAN_FRANCISCO',), array([0.0005456]))
73007
(('__label__LOS_ANGELES',), array([0.0010013]))
35016
(('__label__SAN_FRANCISCO',), array([0.00054988]))
66711
(('__label__LOS_ANGELES',), array([0.00099638]))
24522
(('__label__SAN_FRANCISCO',), array([0.00054338]))
54612
(('__label__LOS_ANGELES',), array([0.00104909]))
87820
(('__label__SAN_FRANCISCO',), array([0.000595]))
31712
(('__label__SAN_FRANCISCO',), array([0.00059742]))
27007
(('__label__SAN_FRANCISCO',), array([0.00059999]))
82510
(('__label__SAN_FRANCISCO',), array([0.0006633]))
98602
(('__label__LOS_ANGELES',), array([0.00256669]))
80802
(('__label__SAN_FRANCISCO',), array([0.00066714]))
91007
(('__label__LOS_ANGELES',), array([0.00103501]))
95521
(('__label__SAN_FRANCISCO',), array([0.00058563]))
83212
(('__label__SAN_FRANCISCO',), array([0.0005456]))
51231
(('__label__SAN_FRANCISCO',), array([0.00064894]))
82510
(('__label__SAN_FRANCISCO',), array([0.00

(('__label__SAN_FRANCISCO',), array([0.00056867]))
82510
(('__label__SAN_FRANCISCO',), array([0.0006633]))
24522
(('__label__SAN_FRANCISCO',), array([0.00054338]))
71001
(('__label__LOS_ANGELES',), array([0.00101985]))
68816
(('__label__SAN_FRANCISCO',), array([0.00063586]))
87820
(('__label__SAN_FRANCISCO',), array([0.000595]))
13020
(('__label__SAN_FRANCISCO',), array([0.0005507]))
68815
(('__label__LOS_ANGELES',), array([0.00099086]))
81121
(('__label__SAN_FRANCISCO',), array([0.00056331]))
83212
(('__label__SAN_FRANCISCO',), array([0.0005456]))
64725
(('__label__SAN_FRANCISCO',), array([0.00056115]))
31712
(('__label__SAN_FRANCISCO',), array([0.00059742]))
83212
(('__label__SAN_FRANCISCO',), array([0.0005456]))
71001
(('__label__LOS_ANGELES',), array([0.00101985]))
24522
(('__label__SAN_FRANCISCO',), array([0.00054338]))
63621
(('__label__LOS_ANGELES',), array([0.00103065]))
81121
(('__label__SAN_FRANCISCO',), array([0.00056331]))
54612
(('__label__LOS_ANGELES',), array([0.00104909

In [15]:
model_name="tax_"+"1_"
model_flight.save_model(model_name+".bin")

# Flight Training

In [82]:
total_error_to_repaired=0
total_repaired=0
for fds in  flight_fds:
    train_data_rows=[]
    data_for_retrain=retrain_data[fds]
    data_for_retrain[fds[1]]='__label__' + data_for_retrain[fds[1]].astype(str)
    data_for_retrain[fds[1]]=[s.replace(" ", "_") for s in data_for_retrain[fds[1]]]
    df3 = data_for_retrain[data_for_retrain[fds[1]] != "__label__None"]
    #model = fasttext.train_supervised(input=df3)
    df3.to_csv(r'datasets/training_flight.txt', index=False, sep=' ', header=False)
    ! wc datasets/training_flight.txt
    #! head -n 819  datasets/training_flight.txt > datasets/flight.train
    #! tail -n 200  datasets/training_flight.txt > datasets/flight.valid
    model_flight=fasttext.train_supervised(input="datasets/training_flight.txt",epoch=100)
    error_correction=prepare_testing_datasets_real_world_data_error(dataframe,clean_dataframe,"fds",dataset_name,fds)
    #model_flight.test("datasets/flight.valid")
    #print_results(*model_flight.test('datasets/flight.valid'))
    for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
        predict_value=retrain_data.at[index, fds[0]]
        
       # total_error_to_repaired=total_error_to_repaired+1
            
        repaired=model_flight.predict(predict_value,k=1,threshold=0.5)
        try:
            if repaired[0][0]:
                total_error_to_repaired=total_error_to_repaired+1   
                k=repaired[0][0].replace('__label__','')
                repaired_k=k.replace('_',' ')
                print(repaired_k,actual_value)
                repaired_k=repaired_k.strip()
                actual_value=actual_value.strip()
                if repaired_k==actual_value:
                    total_repaired=total_repaired+1
        except:
            print("Threshold")
print(total_error,total_error_to_repaired,total_repaired )
if total_error_to_repaired>0:
    p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
else:
    p,r,f="Invalid", "Invalid", "Invalid"
print(p,r,f)

            
        
    

    

  818  1636 28355 datasets/training_flight.txt


/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


11:55 a.m. 11:55 a.m.
2:55 p.m. 1:30 p.m.
2:55 p.m. 8:25 p.m.
2:55 p.m. 9:00 p.m.
Threshold
2:55 p.m. 11:55 p.m.
Threshold
Threshold
2:55 p.m. 11:25 p.m.
3:27 p.m. 3:27 p.m.
Threshold
2:55 p.m. 6:30 p.m.
3:58 p.m. 3:58 p.m.
10:54 a.m. 10:54 a.m.
7:31 a.m. 7:31 a.m.
7:22 a.m. 7:22 a.m.
8:39 p.m. 8:39 p.m.
Threshold
2:55 p.m. 11:08 p.m.
2:55 p.m. 1:30 p.m.
2:55 p.m. 8:25 p.m.
2:55 p.m. 9:00 p.m.
Threshold
1:48 p.m. 1:48 p.m.
1:31 p.m. 1:31 p.m.
4:28 p.m. 4:28 p.m.
7:30 a.m. 7:30 a.m.
9:14 a.m. 9:14 a.m.
Threshold
12:11 p.m. 12:11 p.m.
6:55 p.m. 6:55 p.m.
2:55 p.m. 6:30 p.m.
8:25 p.m. 8:25 p.m.
5:22 p.m. 5:22 p.m.
Threshold
2:26 p.m. 2:26 p.m.
12:35 p.m. 12:35 p.m.
12:41 p.m. 12:41 p.m.
10:18 a.m. 10:18 a.m.
8:04 a.m. 8:04 a.m.
11:30 a.m. 11:30 a.m.
11:56 a.m. 11:56 a.m.
12:12 p.m. 12:12 p.m.
7:29 a.m. 7:29 a.m.
Threshold
2:55 p.m. 11:25 p.m.
7:58 p.m. 7:58 p.m.
8:42 a.m. 8:42 a.m.
7:14 p.m. 7:14 p.m.
7:31 a.m. 7:31 a.m.
7:39 a.m. 7:39 a.m.
6:10 a.m. 6:10 a.m.
3:58 p.m. 3:58 p.m.
6:57 a.m

9:14 a.m. 9:14 a.m.
Threshold
12:11 p.m. 12:11 p.m.
11:05 a.m. 11:05 a.m.
10:10 a.m. 10:10 a.m.
3:31 p.m. 3:31 p.m.
12:35 p.m. 12:35 p.m.
10:18 a.m. 10:18 a.m.
6:08 a.m. 6:08 a.m.
2:48 p.m. 2:48 p.m.
7:25 a.m. 7:25 a.m.
10:19 a.m. 10:19 a.m.
7:29 a.m. 7:29 a.m.
12:41 a.m. 12:41 a.m.
5:56 a.m. 5:56 a.m.
12:13 p.m. 12:13 p.m.
8:42 a.m. 8:42 a.m.
2:55 p.m. 6:30 p.m.
11:55 a.m. 11:55 a.m.
2:55 p.m. 2:55 p.m.
Threshold
2:55 p.m. 11:08 p.m.
2:55 p.m. 1:30 p.m.
2:55 p.m. 8:25 p.m.
2:55 p.m. 9:00 p.m.
Threshold
2:55 p.m. 11:55 p.m.
Threshold
Threshold
2:55 p.m. 6:30 p.m.
Threshold
2:55 p.m. 2:55 p.m.
Threshold
2:55 p.m. 11:25 p.m.
3:27 p.m. 3:27 p.m.
7:58 p.m. 7:58 p.m.
9:14 a.m. 9:14 a.m.
7:21 a.m. 7:21 a.m.
8:23 p.m. 8:23 p.m.
2:55 p.m. 11:25 p.m.
6:57 a.m. 6:57 a.m.
2:55 p.m. 6:30 p.m.
3:27 p.m. 3:27 p.m.
7:14 p.m. 7:14 p.m.
4:29 p.m. 4:29 p.m.
11:01 a.m. 11:01 a.m.
12:12 p.m. 12:12 p.m.
2:55 p.m. 8:25 p.m.
2:04 p.m. 2:04 p.m.
12:35 p.m. 12:35 p.m.
7:39 a.m. 7:39 a.m.
6:08 a.m. 6:08 a.m.
4:

 1465  2930 50808 datasets/training_flight.txt


...

10:45 a.m. 10:45 a.m.
6:40 a.m. 6:40 a.m.
7:05 p.m. 7:05 p.m.
12:55 p.m. 12:55 p.m.
6:17 p.m. 6:17 p.m.
3:30 p.m. 3:30 p.m.
6:40 p.m. 6:40 p.m.
6:30 p.m. 6:30 p.m.
2:35 p.m. 2:35 p.m.
2:15 p.m. 2:15 p.m.
11:35 a.m. 11:35 a.m.
5:55 p.m. 5:55 p.m.
4:05 p.m. 4:05 p.m.
9:20 p.m. 9:20 p.m.
6:40 p.m. 6:40 p.m.
5:11 a.m. 5:11 a.m.
3:50 p.m. 3:50 p.m.
11:50 a.m. 11:50 a.m.
10:34 a.m. 10:34 a.m.
11:50 p.m. 11:50 p.m.
12:15 a.m. 12:15 a.m.
12:29 a.m. 12:29 a.m.
4:17 p.m. 4:17 p.m.
4:31 p.m. 4:31 p.m.
2:59 p.m. 2:59 p.m.
9:45 p.m. 9:45 p.m.
6:23 p.m. 6:23 p.m.
8:05 p.m. 8:05 p.m.
9:35 a.m. 9:35 a.m.
7:30 a.m. 7:30 a.m.
12:18 p.m. 12:18 p.m.
12:21 a.m. 12:21 a.m.
3:15 p.m. 3:15 p.m.
9:10 p.m. 9:10 p.m.
7:25 p.m. 7:25 p.m.
10:35 a.m. 10:35 a.m.
4:00 p.m. 4:00 p.m.
2:45 p.m. 2:45 p.m.
2:25 p.m. 2:25 p.m.
8:56 p.m. 8:56 p.m.
11:10 a.m. 11:10 a.m.
8:22 p.m. 8:22 p.m.
1:45 p.m. 1:45 p.m.
6:15 p.m. 6:15 p.m.
3:19 p.m. 3:19 p.m.
2:45 p.m. 2:45 p.m.
7:45 a.m. 7:45 a.m.
2:35 p.m. 2:35 p.m.
8:00 a.m. 8:00 a

4:53 p.m. 4:53 p.m.
9:20 p.m. 9:20 p.m.
1:40 p.m. 1:40 p.m.
8:27 a.m. 8:27 a.m.
5:11 a.m. 5:11 a.m.
9:43 a.m. 9:43 a.m.
4:48 p.m. 4:48 p.m.
9:24 a.m. 9:24 a.m.
12:15 a.m. 12:15 a.m.
12:29 a.m. 12:29 a.m.
5:50 a.m. 5:50 a.m.
4:17 p.m. 4:17 p.m.
10:31 a.m. 10:31 a.m.
6:44 p.m. 6:44 p.m.
7:35 p.m. 7:35 p.m.
8:05 p.m. 8:05 p.m.
7:53 p.m. 7:53 p.m.
12:21 a.m. 12:21 a.m.
4:00 p.m. 4:00 p.m.
5:15 p.m. 5:15 p.m.
8:56 p.m. 8:56 p.m.
8:22 p.m. 8:22 p.m.
2:00 p.m. 2:00 p.m.
3:19 p.m. 3:19 p.m.
1:39 p.m. 1:39 p.m.
10:54 a.m. 10:54 a.m.
10:20 a.m. 10:20 a.m.
1:17 p.m. 1:17 p.m.
5:00 p.m. 5:00 p.m.
11:35 a.m. 11:35 a.m.
7:03 p.m. 7:03 p.m.
2:20 p.m. 2:20 p.m.
10:06 a.m. 10:06 a.m.
1:16 p.m. 1:16 p.m.
9:21 a.m. 9:21 a.m.
5:35 a.m. 5:35 a.m.
7:23 a.m. 7:23 a.m.
6:55 a.m. 6:55 a.m.
11:19 a.m. 11:19 a.m.
2:50 p.m. 2:50 p.m.
9:54 a.m. 9:54 a.m.
10:15 p.m. 10:15 p.m.
4:49 p.m. 4:49 p.m.
5:25 p.m. 5:25 p.m.
10:40 a.m. 10:40 a.m.
11:53 a.m. 11:53 a.m.
11:45 a.m. 11:45 a.m.
7:03 p.m. 7:03 p.m.
4:53 p.m. 4:53

/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel-4.10.0-py3.5.egg/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Threshold
Threshold
Threshold
Threshold
Threshold
3:30 p.m. 3:30 p.m.
6:55 a.m. 6:55 a.m.
Threshold
7:25 p.m. 7:25 p.m.
Threshold
11:12 a.m. 11:12 a.m.
6:19 a.m. 6:19 a.m.
6:36 p.m. 6:36 p.m.
12:50 p.m. 12:50 p.m.
5:38 p.m. 5:38 p.m.
3:14 p.m. 3:14 p.m.
6:15 p.m. 6:15 p.m.
6:43 p.m. 6:43 p.m.
3:13 p.m. 3:13 p.m.
1:43 p.m. 1:43 p.m.
11:06 a.m. 11:06 a.m.
5:43 p.m. 5:43 p.m.
3:43 p.m. 3:43 p.m.
9:20 p.m. 9:20 p.m.
Threshold
Threshold
Threshold
11:44 a.m. 11:44 a.m.
10:04 a.m. 10:04 a.m.
Threshold
Threshold
Threshold
4:17 p.m. 4:17 p.m.
4:23 p.m. 4:23 p.m.
3:04 p.m. 3:04 p.m.
9:53 p.m. 9:53 p.m.
6:38 p.m. 6:38 p.m.
7:55 p.m. 7:55 p.m.
9:31 a.m. 9:31 a.m.
7:51 a.m. 7:51 a.m.
12:09 p.m. 12:09 p.m.
Threshold
3:16 p.m. 3:16 p.m.
8:53 p.m. 8:53 p.m.
10:36 a.m. 10:36 a.m.
3:33 p.m. 3:33 p.m.
2:46 p.m. 2:46 p.m.
2:27 p.m. 2:27 p.m.
Threshold
11:06 a.m. 11:06 a.m.
8:03 p.m. 8:03 p.m.
1:15 p.m. 1:15 p.m.
5:42 p.m. 5:42 p.m.
3:21 p.m. 3:21 p.m.
2:50 p.m. 2:50 p.m.
7:40 a.m. 7:40 a.m.
3:30 p.m. 3:30

3:30 p.m. 3:30 p.m.
6:55 a.m. 6:55 a.m.
Threshold
12:09 p.m. 12:09 p.m.
10:36 a.m. 10:36 a.m.
9:53 p.m. 9:53 p.m.
6:55 a.m. 6:55 a.m.
7:40 a.m. 7:40 a.m.
7:25 p.m. 7:25 p.m.
5:43 p.m. 5:43 p.m.
10:29 p.m. 10:29 p.m.
7:39 p.m. 7:39 p.m.
3:04 p.m. 3:04 p.m.
4:09 p.m. 4:09 p.m.
Threshold
3:30 p.m. 3:30 p.m.
3:21 p.m. 3:21 p.m.
11:06 a.m. 11:06 a.m.
9:05 a.m. 9:05 a.m.
7:39 p.m. 7:39 p.m.
10:04 a.m. 10:04 a.m.
2:46 p.m. 2:46 p.m.
11:12 a.m. 11:12 a.m.
6:19 a.m. 6:19 a.m.
7:40 a.m. 7:40 a.m.
12:50 p.m. 12:50 p.m.
3:14 p.m. 3:14 p.m.
3:13 p.m. 3:13 p.m.
1:43 p.m. 1:43 p.m.
8:45 a.m. 8:45 a.m.
Threshold
11:06 a.m. 11:06 a.m.
3:43 p.m. 3:43 p.m.
11:32 a.m. 11:32 a.m.
Threshold
Threshold
Threshold
11:44 a.m. 11:44 a.m.
10:04 a.m. 10:04 a.m.
Threshold
Threshold
Threshold
4:17 p.m. 4:17 p.m.
4:23 p.m. 4:23 p.m.
1:40 p.m. 1:40 p.m.
9:53 p.m. 9:53 p.m.
7:36 p.m. 7:36 p.m.
11:06 a.m. 11:06 a.m.
7:55 p.m. 7:55 p.m.
9:31 a.m. 9:31 a.m.
7:51 a.m. 7:51 a.m.
12:09 p.m. 12:09 p.m.
3:16 p.m. 3:16 p.m.
8:53

In [17]:
def evaluate_model (total_error, total_error_to_repair, total_correction):
        if total_error_to_repair==0:
            precision=0.00
        else:
            precision=total_correction/total_error_to_repair
            precision=round(precision,2)
        if total_error==0:
            recall=0.00
        else:
            recall=total_correction/total_error
            recall=round(recall,2)
        if (precision+recall)==0:
            f_score=0.00
        else:
            f_score=(2 * precision * recall) / (precision + recall) 
            f_score=round(f_score,2)     
        return precision, recall,f_score

In [72]:
k=repaired[0][0].replace('__label__','')
k.replace('_',' ')

'6:55 a.m.'

In [39]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [50]:
K=(('__label__11:55_a.m.',), array([0.9977563]))

NameError: name 'array' is not defined